# **Seminararbeit**
von Leon Lantz

## **🚀 Bibliotheken-Import und CUDA-Verfügbarkeit**

In [1]:
import torch
import os
from diffusers import StableDiffusionPipeline
from diffusers import DiffusionPipeline
import shutil
import json
import mediapipe as mp
from torch import autocast
import random
import cv2
from pycocotools.coco import COCO

print("Cuda verfügbar? -->", torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

c:\Users\leonl\anaconda3\envs\stable_diffusion\lib\site-packages\diffusers\models\transformers\transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


Cuda verfügbar? --> True
NVIDIA GeForce RTX 4070 Ti


## **🖼️ COCO Dataset** 
https://cocodataset.org/

- mehr als 200.000 reale Bilder aus unterschiedlichsten Szenarien
- unterteilt in 80 Kategorien (Personen, Fahrzeuge, Tiere, ...)
- jedes Bild detailliert annotiert, unter anderem mit Bildunterschriften (Captions)


In [43]:
# Ordner-Pfade
results_path  = 'results/'
images_path = 'coco_dataset/train2017'
annotations_path = 'coco_dataset/annotations/instances_train2017.json'
captions_path = 'coco_dataset/annotations/captions_train2017.json'

In [11]:
# Lade die COCO-Annotationen
with open(annotations_path, 'r') as f:
    coco_annotations = json.load(f)

# Lade die COCO-Captions
with open(captions_path, 'r') as f:
    coco_captions = json.load(f)


# Erstelle ein Mapping für ID und Bild-Pfad
mapping_filename = {}
for image in coco_annotations['images']:
    mapping_filename[image['id']] = image['file_name']

def get_filename_from_image_id(image_id):
    return mapping_filename.get(image_id, "Bild-ID nicht gefunden")


# Erstelle ein leeres Dictionary für das Mapping
image_id_to_caption = {}

# Durchlaufe die Liste der Captions und fülle das Mapping
for annotation in coco_captions['annotations']:
    image_id = annotation['image_id']
    caption = annotation['caption']
    if image_id not in image_id_to_caption:
        image_id_to_caption[image_id] = [caption]
    else:
        image_id_to_caption[image_id].append(caption)

def get_caption_from_image_id(image_id):
    captions_list = image_id_to_caption.get(image_id, "Bild-ID nicht gefunden")
    
    # Sortiere die Bildunterschriften nach der Länge der Wörter
    sorted_captions = sorted(captions_list, key=lambda x: len(x.split()), reverse=True)
    
    # Gib die erste (längste) Bildunterschrift zurück
    return sorted_captions[0]

In [45]:
# Lade die COCO-Annotationen
with open(annotations_path, 'r') as f:
    coco_annotations = json.load(f)

# Lade die COCO-Captions
with open(captions_path, 'r') as f:
    coco_captions = json.load(f)


In [51]:
print(coco_annotations['images'][0])
print(coco_captions["annotations"][0])

{'license': 3, 'file_name': '000000391895.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg', 'height': 360, 'width': 640, 'date_captured': '2013-11-14 11:18:45', 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg', 'id': 391895}
{'image_id': 203564, 'id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.'}


### **🚫 Verwendung des COCO-Helpers um nur Bilder der Kategorie "Person" herauszufiltern**

In [30]:
# Initialisiere COCO-Helper
coco = COCO(annotations_path)

# Hole alle Kategorien
categories = coco.loadCats(coco.getCatIds())
category_names = [cat['name'] for cat in categories]

# Finde die ID der Kategorie "person" --> Nur Bilder die im Dataset bereits als "Person" markiert
category_id = coco.getCatIds(catNms=['person'])

# Hole alle Annotationen der Kategorie "person"
ann_ids = coco.getAnnIds(catIds=category_id)
anns = coco.loadAnns(ann_ids)

# Hole die Bild-IDs der Annotationen
image_ids = list(set([ann['image_id'] for ann in anns]))

# Filter nochmal nach captions mit bestimmten Worten
words_of_interest = ['person', 'man', 'woman', 'men', 'women', 'kid', 'child', 'face', 'girl', 'boy']

words_of_no_interest = ['ski', 'snow']

def contains_word(sentence, word_list):
    return any(word in sentence for word in word_list)

for id in image_ids:
    caption = get_caption_from_image_id(id)
    if not contains_word(caption, words_of_interest):
        image_ids.remove(id)

def getImageIds(count):
    return random.sample(image_ids, count)

loading annotations into memory...
Done (t=9.60s)
creating index...
index created!


In [42]:
import random
from pycocotools.coco import COCO

# Initialisiere COCO-Helper
coco = COCO(annotations_path)

# Hole alle Kategorien
categories = coco.loadCats(coco.getCatIds())
category_names = [cat['name'] for cat in categories]

# Finde die ID der Kategorie "person" --> Nur Bilder die im Dataset bereits als "Person" markiert
category_id = coco.getCatIds(catNms=['person'])

# Hole alle Annotationen der Kategorie "person"
ann_ids = coco.getAnnIds(catIds=category_id)
anns = coco.loadAnns(ann_ids)

# Hole die Bild-IDs der Annotationen
image_ids = list(set([ann['image_id'] for ann in anns]))

# Filter nochmal nach captions mit bestimmten Worten
words_of_interest = ['person', 'man', 'woman', 'men', 'women', 'kid', 'child', 'face', 'girl', 'boy']
words_of_no_interest = ['ski', 'snow']

def contains_word(sentence, word_list):
    return any(word in sentence for word in word_list)

# Filter image IDs based on captions
filtered_image_ids = []
for image_id in image_ids:
    caption = get_caption_from_image_id(image_id)
    if contains_word(caption, words_of_interest) and not contains_word(caption, words_of_no_interest):
        filtered_image_ids.append(image_id)

def getImageIds(count):
    return random.sample(filtered_image_ids, count)

# Example usage
count = 5  # Number of image IDs you want to retrieve
selected_image_ids = getImageIds(count)
print(selected_image_ids)


loading annotations into memory...
Done (t=12.70s)
creating index...
index created!


KeyError: 'caption'

## **🙂 Gesichtserkennung**

In [5]:
def detectFace(img_name):
    mp_face_detection = mp.solutions.face_detection

    def get_rectangle(objDictionary, ss):
        left = int(objDictionary.xmin * ss[1])
        top = int(objDictionary.ymin * ss[0])
        right = int(left + objDictionary.width * ss[1])
        bottom = int(top + objDictionary.height * ss[0])
        return ((left, top), (right, bottom))

    with mp_face_detection.FaceDetection(model_selection=3, min_detection_confidence=0.8) as face_detection:
        image = cv2.imread(img_name)
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if not results.detections:
        return None

    faces = []

    for obj in results.detections:
        ff = get_rectangle(obj.location_data.relative_bounding_box, image.shape)
        (left, top), (right, bottom) = ff
        faces.append(image[top:bottom, left:right])

    return faces

In [6]:
def prune_detected_faces(folder_name):
  files = os.listdir(folder_name)

  r_count = 0
  for f_name in files:
    faces = detectFace(os.path.join(folder_name,f_name))
    if not faces: 
      os.remove(os.path.join(folder_name,f_name))
      r_count += 1
  
  return r_count

## **✂️ Extrahieren von realen Gesichtern**

In [ ]:
def find_faces(img_path, save_path, num_faces_wanted, face_res = 250):

  if os.path.exists(save_path):
    shutil.rmtree(save_path)
  os.mkdir(save_path)

  count = 0

  for img_id in random.sample(image_ids, num_faces_wanted*10):

    if count >= num_faces_wanted: break

    f_name = get_filename_from_image_id(img_id)

    faces = detectFace(os.path.join(img_path,f_name))
    if not faces: continue

    for face in faces:
      if not face.size: continue
      face = cv2.resize(face,(face_res,face_res))
      cv2.imwrite(os.path.join((f'{save_path}'), str(count)+'.jpeg'), face)
      count += 1      

  return count

In [ ]:
num_runs = 2
num_faces_wanted = 1000
face_res = 100

for n in range(num_runs):
    faces_generated = find_faces(images_path, f'{results_path}/realFaces{n}', num_faces_wanted, face_res) # type: ignore
    print(f'{faces_generated} Gesichter wurden im Ordner realFaces{n} generiert.')
    x = prune_detected_faces((f'{results_path}/realFaces{n}')) # type: ignore
    print(f'{x} Gesichter wurden entfernt.')

## **🔄 Generierung mit Stable Diffusion v1-4**

In [ ]:
def select_model_pipeline(name):
    if name == "stable-diffusion-v1-4":
        # Instanziiere eine Stable Diffusion Pipeline aus dem Modell "CompVis/stable-diffusion-v1-4"
        pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16)  
        pipe.to("cuda")
    elif name == "stable-diffusion-xl-base-1.0":
        # Instanziiere eine Stable Diffusion Pipeline aus dem Modell "CompVis/stable-diffusion-xl-base-1.0"
        pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
        pipe.to("cuda")
    
    return pipe  # Rückgabe der Pipeline am Ende der Funktion

pipe = select_model_pipeline("stable-diffusion-xl-base-1.0")

In [2]:
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.28.2",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [14]:
prompt = "a woman is holding a tennis racket on a tennis court."

images = pipe(prompt=prompt).images[0]
images.save(f"results/test.jpeg")

  0%|          | 0/50 [00:00<?, ?it/s]

In [38]:
def find_faces_generated(save_path_images, save_path_faces, num_faces_wanted, face_res=250):

  if os.path.exists(save_path_images):
    shutil.rmtree(save_path_images)    
  os.mkdir(save_path_images)

  if os.path.exists(save_path_faces):
    shutil.rmtree(save_path_faces)    
  os.mkdir(save_path_faces)

  count_f = 0
  count_i = 0

  for img_id in random.sample(image_ids, num_faces_wanted*10):

    if count_f >= num_faces_wanted: break

    caption = get_caption_from_image_id(img_id)
    prompt = caption.lower()
    neg = "bad anatomy, low quality, ugly, cartoon, anime, bad limbs, bad face, deformed, blurry eyes, multiple fingers, distorted, unrealistic, poorly rendered, unnatural, messy, pixelated, glitch, out of proportion, extra limbs, artifact, strange eyes, poorly drawn, bad perspective, awkward pose, extra legs, low resolution, bad expression, odd lighting, wrong shadows, blurry background, disconnected body parts, unnatural colors"

    print(prompt)

    # generate image
    image = pipe(prompt= prompt, negative_prompt=neg).images[0] 
    image.save(f'{save_path_images}/{count_i}.jpeg')
    count_i += 1

    faces = detectFace(os.path.join(f'{save_path_images}/', str(count_i-1)+'.jpeg'))
    if not faces: continue

    for face in faces:
      if not face.size: continue
      face = cv2.resize(face,(face_res,face_res))
      cv2.imwrite(os.path.join(f'{save_path_faces}/', str(count_f)+'.jpeg'), face)
      count_f += 1     

  return count_i, count_f

In [ ]:
import matplotlib.pyplot as plt

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")

# if using torch < 2.0
# pipe.enable_xformers_memory_efficient_attention()

prompt = "An astronaut riding a green horse"

images = pipe(prompt=prompt).images[0]

# Display the generated image using matplotlib
plt.imshow(images)
plt.axis('off')  # Hide axis
plt.show()

In [ ]:
# Lösche den GPU-Cache
torch.cuda.empty_cache()

In [39]:
num_runs = 1
processed_images = set()
num_faces_wanted = 3
face_res = 250

for n in range(num_runs):
  images_generated, faces_generated = find_faces_generated(f'{results_path}/imagesGenerated{n}', f'{results_path}/facesGenerated{n}', num_faces_wanted, face_res)
  print(f'{faces_generated} faces were generated in folder {results_path}/facesGenerated{n}')
  print(f'{images_generated} faces were generated in folder {results_path}/imagesGenerated{n}')
  x = prune_detected_faces((f'{results_path}/facesGenerated{n}'))
  print(f'{x} faces were removed')

a youth soccer game is being played with a standoff for the ball.


  0%|          | 0/50 [00:00<?, ?it/s]

the two people are walking in the snow with their ski equipment.


  0%|          | 0/50 [00:00<?, ?it/s]

an older man in a business suit who has a beard and glasses. 


...

a motorcyclist and his passenger riding on the highway are reflected in a car's side view mirror.


  0%|          | 0/50 [00:00<?, ?it/s]

a surfer in a wet suit is riding a boats wake


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 